Instalamos e Importamos librerias

In [62]:
#pip install INEGIpy
#pip install sie_banxico

9524.40s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import configparser

In [4]:
# Se requieren tokens proporcionados por el INEGI y Banxico
config = configparser.ConfigParser()
config.read('config.ini')
inegi_token = config['INEGI']['token']
bmx_token = config['BMX']['token']

Importamos inflación del INEGI

In [3]:
from INEGIpy import Indicadores
inegi = Indicadores(inegi_token)

In [4]:
catind = inegi.catalogo_indicadores(banco= "BISE")
catind

,value,Description
0,472079,Total nacional
1,472112,Total nacional
2,472145,Total nacional
3,472178,Total nacional
4,472211,Total nacional
...,...,...
31271,8999999704,56 Servicios de apoyo a los negocios y manejo ...
31272,8999999707,71 Servicios de esparcimiento culturales y dep...
31273,8999999710,53 Servicios inmobiliarios y de alquiler de bi...
31274,8999999711,"54 Servicios profesionales, científicos y tecn..."


In [121]:
#No logré identificar el indicador en el catálogo
#filtered_catind = catind[catind['value'] == 628194]
#print(filtered_catind)

In [65]:
#Guardamos indicador en dataframe y ordenamos un poco
inpc = inegi.obtener_df(indicadores = ["628194"],
                      nombres = "INPC")
inpc.reset_index(inplace=True)
inpc['fechas'] = inpc['fechas'].dt.date
inpc.rename(columns={'fechas': 'fecha'}, inplace=True)
inpc

/home/qnava13/.local/lib/python3.10/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)


,fecha,INPC
0,1969-01-01,0.011595
1,1969-02-01,0.011636
2,1969-03-01,0.011648
3,1969-04-01,0.011680
4,1969-05-01,0.011680
...,...,...
657,2023-10-01,130.609000
658,2023-11-01,131.445000
659,2023-12-01,132.373000
660,2024-01-01,133.555000


In [69]:
#Creamos csv
inpc.to_csv('datos/inpc.csv', index=False)

Importamos tipo de cambio y tasa de interés de BANXICO

In [5]:
from sie_banxico  import SIEBanxico
api_banxico = SIEBanxico(token = bmx_token, id_series = ['SF17906' ,'SF282'], language = 'es')

In [6]:
#Obtenemos datos y transformamos en dataframedata
data_banxico = api_banxico.get_timeseries(pct_change='PorcAnual')

In [7]:
df_banxico = pd.json_normalize(data_banxico['bmx']['series'], record_path = 'datos', meta = ['idSerie', 'titulo'])

In [16]:
df_banxico['fecha'] = pd.to_datetime(df_banxico['fecha'], format='%d/%m/%Y')
df_banxico

,fecha,dato,idSerie,titulo
0,1979-01-01,N/E,SF282,Cetes 28 días Tasa de rendimiento promedio men...
1,1979-02-01,N/E,SF282,Cetes 28 días Tasa de rendimiento promedio men...
2,1979-03-01,N/E,SF282,Cetes 28 días Tasa de rendimiento promedio men...
3,1979-04-01,N/E,SF282,Cetes 28 días Tasa de rendimiento promedio men...
4,1979-05-01,N/E,SF282,Cetes 28 días Tasa de rendimiento promedio men...
...,...,...,...,...
923,2023-10-01,-9.02,SF17906,Tipo de cambio ...
924,2023-11-01,-10.43,SF17906,Tipo de cambio ...
925,2023-12-01,-13.11,SF17906,Tipo de cambio ...
926,2024-01-01,-8.68,SF17906,Tipo de cambio ...


In [17]:
#Tipo de cambio
tipo_de_cambio = df_banxico[df_banxico['idSerie'] == 'SF17906'][['fecha', 'dato']]
tipo_de_cambio.rename(columns={'dato': 'tipo_de_cambio'}, inplace=True)
tipo_de_cambio = tipo_de_cambio[(tipo_de_cambio['tipo_de_cambio'] != 'N/E') & tipo_de_cambio['tipo_de_cambio'].notnull()]
tipo_de_cambio.reset_index(drop=True, inplace=True)
tipo_de_cambio

,fecha,tipo_de_cambio
0,1992-11-01,1.53
1,1992-12-01,1.39
2,1993-01-01,1.52
3,1993-02-01,1.23
4,1993-03-01,1.04
...,...,...
371,2023-10-01,-9.02
372,2023-11-01,-10.43
373,2023-12-01,-13.11
374,2024-01-01,-8.68


In [18]:
#Guardamos en csv
tipo_de_cambio.to_csv('datos/tipo_de_cambio.csv', index=False)

In [19]:
#Tasa de interes
tasa_de_interes = df_banxico[df_banxico['idSerie'] == 'SF282'][['fecha', 'dato']]
tasa_de_interes.rename(columns={'dato': 'tasa_de_interes'}, inplace=True)
tasa_de_interes = tasa_de_interes[(tasa_de_interes['tasa_de_interes'] != 'N/E') & tasa_de_interes['tasa_de_interes'].notnull()]
tasa_de_interes.reset_index(drop=True, inplace=True)
tasa_de_interes

,fecha,tasa_de_interes
0,1986-02-01,50.21
1,1986-03-01,40.17
2,1986-04-01,39.35
3,1986-05-01,43.54
4,1986-06-01,36.54
...,...,...
452,2023-10-01,24.08
453,2023-11-01,15.61
454,2023-12-01,12.55
455,2024-01-01,6.41


In [20]:
#Guardamos en csv
tasa_de_interes.to_csv('datos/tasa_de_interes.csv', index=False)

Load

In [21]:
#Load# Abres un cliente de S3
import boto3
session = boto3.Session(profile_name='arquitectura')
s3 = session.client('s3')

In [22]:
#Upload
BUCKET_NAME = "itam-analytics-enriquenava"
s3.upload_file(Filename="datos/inpc.csv", Bucket=BUCKET_NAME, Key="tarea6/raw/inflacion/inpc.csv")
s3.upload_file(Filename="datos/tasa_de_interes.csv", Bucket=BUCKET_NAME, Key="tarea6/raw/tasainteres/tasa_de_interes.csv")
s3.upload_file(Filename="datos/tipo_de_cambio.csv", Bucket=BUCKET_NAME, Key="tarea6/raw/tipocambio/tipo_de_cambio.csv")